In [ ]:
import os  # system functions
import os.path as op
from glob import glob
from pathlib import Path

import pandas as pd, numpy as np
# from functools import partial

from nipype import config
# config.enable_provenance()

import nipype.interfaces.spm as spm
import nipype.interfaces.spm.utils as spmu
from nipype.interfaces.dcm2nii import Dcm2nii

import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.utility as util  # utility
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.algorithms.rapidart as ra  # artifact detection
import nipype.algorithms.modelgen as model  # model specification
import nipype.interfaces.matlab as mlab

# TODO
### BIDS-compatible naming
### automated MATLAB and SPM pathfinding
### model building

In [ ]:
# GLOBALS

teeone = 'T1'


In [ ]:
# Prompt for path
spm_path = ''
matlab_path = ''

if not spm.SPMCommand().version:    
    while not op.isdir(spm_path):
        spm_path = str(input('Please provide your SPM path:\t'))
        if not op.isdir(spm_path):
            print('That\'s not a valid path, let\'s try again.')
    while not op.isfile(matlab_path):
        #print(matlab_path, '\n', op.isfile(matlab_path), '\n')
        matlab_path = str(input('Please provide your Matlab path:\t'))
        if not op.isfile(matlab_path):
            print('That\'s not a valid path, let\'s try again.')

    matlab_path += ' -nodesktop -nosplash'

#setup paths to SPM and matlab
#if preferred, you can hardcode your SPM and matlab paths here
spm.SPMCommand.set_mlab_paths(paths=spm_path, 
                              matlab_cmd=matlab_path)


In [ ]:
# DEFINE BACKEND FUNCTIONS
#Assign spm procedures to preprocessing steps

di = spmu.DicomImport()
# use NewSegment in NiPyPe, Segment refers to an old routine (before SPM8)
seg = spm.NewSegment()
realign = spm.Realign()
coreg = spm.Coregister()
# Use Normalize12 as it corresponds to SPM12 routine
norm12 = spm.Normalize12()
smth = spm.Smooth()

In [ ]:
# DEFINE FUNCTIONS

def conversion(input_dir, output_dir):
    di.inputs.in_files = input_dir
    di.inputs.output_dir = output_dir
    di.run() 
    
    
def segmentation(path_structural, deformation_fields=[False, True]):
    
    # Segmentation of T1 data, does not depend on functional files.
    # Segment, Volumes = T1, Deformation field = forward (default)

    #tissue maps
    #path to TMP.nii
    tpm_file = glob(op.join(spm_path, '**', 'tpm.nii'))[0]
    tissue1 = ((tpm_file, 1), 2, (True, True), (False, False))
    tissue2 = ((tpm_file, 2), 2, (True, True), (False, False))
    tissue3 = ((tpm_file, 3), 2, (True, False), (False, False))
    tissue4 = ((tpm_file, 4), 2, (False, False), (False, False))
    tissue5 = ((tpm_file, 5), 2, (False, False), (False, False))
    seg.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5]
    
    seg.inputs.channel_files = path_structural
    seg.inputs.write_deformation_fields = deformation_fields #Which deformation fields to write:[Inverse, Forward]
    # ADD tissue maps
    seg.run()
    

def realignment(input_files, rtm = True, jobtype = 'estwrite', quality = 0.9, fwhm = 5,
                separation =4, interp = 2, wrap = [0, 0, 0], w_which = [2, 1],
                w_interp = 4, w_wrap = [0, 0, 0], w_mask = True, prefix = 'r' ):
    
    # 1. REALIGNMENT
    # Default: Estimate & Reslice, OUTPUT = realigned files (^.r)   
    
    realign.inputs.in_files = input_files
    realign.inputs.register_to_mean = rtm
    realign.inputs.jobtype = jobtype
    realign.inputs.quality = quality
    realign.inputs.fwhm = fwhm
    realign.inputs.separation = separation
    realign.inputs.interp = interp 
    realign.inputs.wrap = wrap

    realign.inputs.write_which = w_which
    realign.inputs.write_interp = w_interp
    realign.inputs.write_wrap = w_wrap
    realign.inputs.write_mask = w_mask
    realign.inputs.out_prefix = prefix

    realign.run()
    
    
def coregistration(path_mean_functional, path_structural, jobtype = 'estimate', cost_f = 'nmi',
                   fwhm = [7., 7.], separation = [4., 2.],
                   tolerance = [0.02, 0.02, 0.02, 0.001, 0.001, 0.001, 0.01, 0.01, 0.01, 0.001, 0.001, 0.001]):
    
    # 2. COREGISTRATION
    # Default: Coregister(Estimate), reference image = mean fMRI, source image = T1 

    coreg.inputs.target = path_mean_functional #mean functional image
    coreg.inputs.source = path_structural #structural image
    coreg.inputs.jobtype = jobtype
    coreg.inputs.cost_function = cost_f
    coreg.inputs.fwhm = fwhm
    coreg.inputs.separation = separation
    coreg.inputs.tolerance = tolerance

    coreg.run()
    
    
def normalisation(path_y_s, path_realigned, jobtype = 'write', w_voxel_sizes = [3., 3., 3.],
                  w_bounding_box = [[-78., -112.,  -70.],[78., 76., 85]], w_interp = 4,
                  out_prefix = 'w'):
    # 4. NORMALISATION
    # Normalise(Write)
    # Deformation field = y_s(T1)
    # Images to write = realigned files (^r*)
    # Default voxel size = [3 3 3]

    norm12.inputs.deformation_file = path_y_s #deformation field
    norm12.inputs.apply_to_files = path_realigned #functional realigned
    norm12.inputs.jobtype = jobtype
    norm12.inputs.write_voxel_sizes = w_voxel_sizes
    norm12.inputs.write_bounding_box = w_bounding_box
    norm12.inputs.write_interp = w_interp
    norm12.inputs.out_prefix = out_prefix

    norm12.run()    

                                        
def smoothing(path_normalised, fwhm = [6., 6., 6.], imp_masking = False, out_prefix = 's'):
                                        
    # 5. SMOOTHING
    # Smooth, images to smooth = normalised files (^wr*)
    # FWHM at least twice voxel size, default = [6 6 6]

    smth.inputs.in_files = path_normalised
    smth.inputs.fwhm = fwhm
    smth.inputs.implicit_masking = imp_masking
    smth.inputs.out_prefix = out_prefix

    smth.run()
                                        

In [ ]:
# create an object with the commont path prefix
prefix = os.getcwd()

# extract list of study participants
dirs_participants = [p for p in glob(op.join(prefix, '*')) if op.isdir(p)] #get all participants from a study
participants = list(set([op.split(f)[-1] for f in dirs_participants])) #list with all participants from a study

# extract list of timepoints
dirs_tps = [p for p in glob(op.join(prefix, '*', '*'), recursive = True ) if op.isdir(p)]
tps = list(set([op.split(f)[-1] for f in dirs_tps])) #list with all timepoints

# extract list of tasks
dirs_tasks = [p for p in glob(op.join(prefix, '*', '*', 'DICOM', '*'), recursive = True ) if op.isdir(p)]
tasks =  list(set([op.split(f)[-1].split('-')[1] for f in dirs_tasks]))


In [ ]:
# create a nested dictionaty - data structure containing all pathnames
# sorta like the old multi-index dataframe, but easier to navigate
# Dictionary structure
# dict_study: {participant: {timepoints: {tasks: {dicom_dir, nii_dir, tasktyoe}}}}

dict_study = {participant:{} for participant in participants}

# set to keep tasknames
tasknames = set()

keys = ['dicom_dir', 'nii_dir', 'tasktype']

for participant in dict_study.keys():
    for tp in tps:
        dict_study[participant][tp] = {}
        for task in tasks:
            # CAUTION! Hardcoded: only a single DICOM dir per task. Needs improvement for multiple dirs present
            dicom_dir = glob(op.join(prefix, participant, tp, 'DICOM', '*' + task + '*'), recursive = True)[0]
            tasktype = task.split('_')[0]
            # CAUTION! Hardcoded: previous knowledge about tasknames and procedure names.
            # Needs improvement for better encoding of names
            if 't1' in tasktype.lower():
                taskname = teeone
            else:
                taskname = task.split(tasktype)[1][6:]
            
            tasknames.add(taskname)
            
            nii_dir = op.join(prefix, participant, tp, 'NIFTI', taskname)
            dict_study[participant][tp][taskname] = {
                keys[0] : dicom_dir, keys[1] : nii_dir, keys[2] : tasktype}

tasknames = list(tasknames) 

In [ ]:
# # Paths to nifti files
# not needed since they are defined in the dict_study population

# cols = ['Participant', 'Timepoint', 'Task', 'Filepath']
# fnii_paths = pd.DataFrame(columns = cols)
# fstruct_paths =  pd.DataFrame(columns = cols)

# timepoints = []
# fMRI_tasks = []

# for participant in dirs_participants:
#     dirs_timepoints = [p for p in glob(op.join(participant, '*')) if op.isdir(p)]
#     tps = list(set([op.split(f)[-1] for f in dirs_timepoints]))
#     participant_name = op.split(participant)[-1].split('_')[1]
#     for tp in tps:
#         fls_tasks = [p for p in glob(op.join(participant, tp, 'NIFTI', '*')) if op.isdir(p)]
#         tasks = tps = list(set([op.split(f)[-1] for f in fls_tasks]))
#         if (tp not in timepoints):
#             timepoints.append(tp)
            
#         for task in tasks:
#             f_path = op.join(participant, tp, 'NIFTI', task)
#             row = [participant_name, tp, task, f_path]
#             if (task not in fMRI_tasks):
#                 fMRI_tasks.append(task)
                
#             if any( t_one in task for t_one in ('t1', 'T1')):
#                 fstruct_paths = fstruct_paths.append(pd.DataFrame([row], columns = cols), ignore_index = True)
#             else:
#                 fnii_paths = fnii_paths.append(pd.DataFrame([row], columns = cols), ignore_index = True)


In [ ]:
#################################
#
# CONVERT DICOM to .NII
#
#################################

             
for participant in participants:
    for tp in tps:
        for task in tasknames:
            nii_path = dict_study[participant][tp][task][keys[1]]
            # if a folder already exists
            if op.isdir(nii_path):
                # and is not empty
                if len(os.listdir(nii_path)) > 0:
                    ask = True
                    while ask:
                        #ask if files should be overwritten
                        print('\n', nii_path)
                        q = str.lower(input('Files detected in target directory. Overwrite? - Y/N:\t'))
                        if any( answer == q for answer in ('y', 'yes', 'n', 'no')):
                            ask = False
                        if any( answer == q for answer in ('y', 'yes')):
                            #convert
                            conversion(glob(op.join(dict_study[participant][tp][task][keys[0]], '**', '*.dcm'), recursive = True),
                                       nii_path) 
            # if a folder does not exist
            else:
                out_dir = Path(nii_path)
                out_dir.mkdir(parents = True)
                conversion(glob(op.join(dict_study[participant][tp][task][keys[0]], '**', '*.dcm'), recursive = True),
                           nii_path)                       

In [ ]:
##################################################
#
#  PREPROCESSING step by step
#
######################################

# 3. SEGMENTATION
# Segmentation of T1 data, does not depend on functional files.
# Done once per timepoint
#
# Segment, Volumes = T1, Deformation field = forward
# use NewSegment in NiPyPe, Segment refers to an old routine (before SPM8)


for participant in participants:
    for tp in tps:
        # path to the structural file
        segmentation(glob(op.join(dict_study[participant][tp][teeone][keys[1]], 's*.nii')))

In [ ]:
# 1. REALIGNMENT
# Estimate & Reslice, OUTPUT = realigned files (^.r)

for participant in participants:
    for tp in tps:
        for task in tasknames:
            if task != teeone:
                realignment(glob(op.join(dict_study[participant][tp][task][keys[1]], 'f*.nii'))[0])


In [ ]:
# 2. COREGISTRATION
# Coregister(Estimate), reference image = mean fMRI, source image = T1for participant in participants:

for tp in tps:
    for task in tasknames:
        if task != teeone:
            coregistration(glob(op.join(dict_study[participant][tp][task][keys[1]], 'mean*.nii'))[0], #mean functional image
                           glob(op.join(dict_study[participant][tp][teeone][keys[1]], 's*.nii'))[0]) #structural image
                               

In [ ]:
# 4. NORMALISATION
# Normalise(Write)
# Deformation field = y_s(T1)
# Images to write = realigned files (^r*)
# Voxel size = [3 3 3]for participant in participants:

for tp in tps:
    for task in tasknames:
        if task != teeone:
            normalisation(glob(op.join(dict_study[participant][tp][teeone][keys[1]], 'y_s*.nii'))[0], #deformation field
                          glob(op.join(dict_study[participant][tp][task][keys[1]], 'rf*.nii'))) #functional realigned


In [ ]:
# 5. SMOOTHING
# Smooth, images to smooth = normalised files (^wr*)
# FWHM at least twice voxel size

for participant in participants:
    for tp in tps:
        for task in tasknames:
            if task != teeone:
                smoothing(glob(op.join(dict_study[participant][tp][task][keys[1]], 'wrf*.nii')))


In [ ]:
###################################################
#
#  PREPROCESSING in one go
#
######################################

for participant in participants:
    for tp in tps:
        segmentation(glob(op.join(dict_study[participant][tp][teeone][keys[1]], 's*.nii')))
        for task in tasknames:
            if task != teeone:
                realignment(glob(op.join(dict_study[participant][tp][task][keys[1]], 'f*.nii'))[0])
                coregistration(glob(op.join(dict_study[participant][tp][task][keys[1]], 'mean*.nii'))[0], #mean functional image
                               glob(op.join(dict_study[participant][tp][teeone][keys[1]], 's*.nii'))[0]) #structural image
                normalisation(glob(op.join(dict_study[participant][tp][teeone][keys[1]], 'y_s*.nii'))[0], #deformation field
                              glob(op.join(dict_study[participant][tp][task][keys[1]], 'rf*.nii'))) #functional realigned
                smoothing(glob(op.join(dict_study[participant][tp][task][keys[1]], 'wrf*.nii')))
                               